In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [28]:
import os
import re
import gc
import nltk
from nltk.tokenize import RegexpTokenizer
import psutil
import numpy as np
import pandas as pd
from numpy import zeros
from numpy import array
from numpy.random import choice

DATA_DIR = "drive/My Drive/DMPR/data/" # training data

def merge_entity_metadata_by_id():
  entity_ids_df = pd.read_csv(os.path.join(DATA_DIR, "entity_ids.csv"), 
                              usecols=['entity', 'id'],
                              index_col='id',
                             )
  #print(entity_ids_df)
  
  entity_metadata_df = pd.read_csv(os.path.join(DATA_DIR, "entity_metadata.csv"),
                                  )
  entity_metadata_df = entity_metadata_df.rename(columns={'entity':'id', 'humanreadablererpresentation': 'metadata'})
  #print(entity_metadata_df)
  
  # merge data 
  id_entity_metadata_df = pd.merge(entity_ids_df,entity_metadata_df, on=['id'], how= 'left')
  print(id_entity_metadata_df.head(10))
  id_entity_metadata_df = id_entity_metadata_df.set_index('entity').filter(regex="[<][a-zA-Z]+[>]", axis=0)
  return id_entity_metadata_df
  
def merge_entity_metatype_by_id():
  entity_types_df = pd.read_csv(os.path.join(DATA_DIR, "entity_types.csv"),
                               )
  entity_types_df = entity_types_df.rename(columns={'entity':'id'})
  #print(entity_types_df)
  
  type_ids_df = pd.read_csv(os.path.join(DATA_DIR, "type_ids.csv"),
                            usecols=['id', 'metatype'],
                           )
  type_ids_df = type_ids_df.rename(columns={'id':'type'})
  #print(type_ids_df)
  
  id_type_metatype_df = pd.merge(entity_types_df,type_ids_df, on=['type'], how= 'left')
  return id_type_metatype_df


# id_entity_metadata_df = merge_entity_metadata_by_id()
# print(id_entity_metadata_df.head(10))

# id_type_metatype_df = merge_entity_metatype_by_id()
# print(id_type_metatype_df.head(5))

# id_entity_metadata_metatype_df = pd.merge(id_type_metatype_df, id_entity_metadata_df, on=['id'], how='left')
# print(id_entity_metadata_metatype_df.head(5))

def merge_news_header_fulltext():
  # get_text_id_header_df
  id_header_file = os.path.join(DATA_DIR, "full-news-100000.txt")
  f = open(id_header_file)
  lines = f.readlines()
  f.close
  id_header_dict = {}
  for line in lines:
    text_id = line.split(",")[0]
    header = line.split(',,"')[-1]
    if text_id not in id_header_dict:
      header = header.rstrip('"\n')
      id_header_dict[text_id] = header
  print(len(id_header_dict))
  id_header_df = pd.DataFrame(list(id_header_dict.items()), columns=['id', 'header'])
  print(id_header_df.head(10))
  
  # get_id_fulltext_df
  id_fulltext_file = os.path.join(DATA_DIR, "text.txt")
  f = open(id_fulltext_file)
  lines = f.readlines()
  f.close()
  id_fulltext_dict = {}
  for index, line in enumerate(lines):
    if index % 2 == 0:
      text_id = line.rstrip("\n")
    elif text_id not in id_fulltext_dict:
      id_fulltext_dict[text_id] = line
  print(len(id_fulltext_dict))
  id_fulltext_df = pd.DataFrame(list(id_fulltext_dict.items()), columns=['id', 'fulltext'])
  print(id_fulltext_df.head(10))
  
  #merge_header_fulltext_by_id

merge_news_header_fulltext()

    

100000
        id                                             header
0  5824417  UAE student-made satellite launching on Novemb...
1  5824418  Here’s what’s on TV this weekend: ‘Sally4Ever’...
2  5824419  Ex-IS captive recalls captivity, seeing her so...
3  5824420  Sri Lanka president dissolves Parliament, call...
4  5824421  Wind respite may help Southern California wild...
5  5824422  Deadly fire leveled a California town in less ...
6  5824423  Pacific Gas & Electric says it will aid wildfi...
7  5824424  Teacher: Bus drivers evacuated students amid C...
8  5824425                    'Hero at Home' by Sarah Verardo
9  5824426  China, US sparring continues over South China Sea
10000
        id                                           fulltext
0  5824417  UAE student-made satellite launching on Novemb...
1  5824418  November 10 at 1:00 AM Listings for Nov. 10. S...
2  5824419  Middle East Ex-IS captive recalls captivity, s...
3  5824420  Email COLOMBO, Sri Lanka – Sri Lanka's Presid

In [0]:


nltk.download('punkt')

########################################## Params ##########################################

EMBED_DIM = 100

# totol news count 10000
START_INDEX = 3500 # start index
END_INDEX = 7000 # trainig number
VALID_START =  7000 # validation start
VALID_END = 8500 #validataion number

# model building
INIT_LR = 0.001 # init learning rate
N_STEPS = 10 
BATCH_SIZE = 16
LAYERS_DIM = [700, 700, 700]
EPOCHS = 15 #training_epochs number

########################################## Data Paths ##########################################

EMBED_DIR = "drive/My Drive/DATA/" # embedding file

training_data = "text.txt" 
glove_embedding = "glove.6B.{:d}d.txt".format(EMBED_DIM)


########################################## Data Prepare ##########################################

"""
read news content from text.txt
split the whole news into sentences
word tokenization
return a list of all tokenized sentences

"""
def file_reader(data_path):
  path = os.path.join(DATA_DIR, data_path)
  file = open(path, encoding='utf-8', mode = "r")
  lines = file.readlines()
  file.close()
  sents = []
  for line in lines:
    # sentence split
    sent_text = nltk.sent_tokenize(line)
      for index, sentence in enumerate(sent_text):
        # word tokenization
        tokenizer = RegexpTokenizer(r'\w+')
        tokenized_sent = tokenizer.tokenize(sentence)
        tokenized_sent = [word.lower() for word in tokenized_sent]
        if index > 0:
          sents.append(tokenized_text)
  return sents

def sents_processing(sents):
  # build new dict to count words in poems
  allWords = {} 
  for sent in sents:
    for word in sent:
      if word not in allWords:
        #print(word)
        allWords[word] = 1
      else:
        allWords[word] = allWords[word]+ 1
  # delete words which less than 2 times used in poems
  erase = []
  for key in allWords:
    if allWords[key] < 5:
        erase.append(key)
  #print(erase)
  allWords['<oov/>'] = 0
  for key in erase:
    allWords['<oov/>'] = allWords['<oov/>']+ allWords[key]
    del allWords[key]
  # sort allWords by counting times: key:word, value:count
  wordPairs = sorted(allWords.items(), key = lambda x: -x[1])
  print(wordPairs)
  words, a= zip(*wordPairs)
  # word to ID, most frequently used word is assigned with a short id
  word2int = dict(zip(words, range(len(words)))) 
#   print(word2int)
  # ID to word
  int2word = dict(zip(word2int.values(), word2int.keys()))
  #print(reverse_dictionary)
  return allWords, word2int, int2word

# return pre_embeddings as word2Vec
def load_glove_embeddings():
  path = os.path.join(DATA_DIR, glove_embedding)
  with open(path) as file:
    #words_list = []
    pre_embeddings = {}
    for line in file:
      l = line.split(" ")
      word_vetor = [float(w) for w in l[1:]]
      pre_embeddings [l[0]] = word_vetor
      #words_list.append(l)
  pre_embeddings ["<oov/>"] = [float(w) for w in np.random.rand(1, EMBED_DIM)[0]]
  return pre_embeddings 

def embedding_processing(dictionary, pre_embeddings):
  embeddings = {}
  words = dictionary.keys()
  for count, word in enumerate(words):
    #print(count," ",pre_embeddings[word])
    if word in pre_embeddings.keys():
      embeddings[count] = pre_embeddings[word]
    elif word.lower() in pre_embeddings.keys():
      embeddings[count] = pre_embeddings[word.lower()]
    else:
#       print("no word--",word)
#       embeddings[count] = pre_embeddings["<oov/>"]
      embeddings[count] = [float(w) for w in np.random.rand(1, EMBED_DIM)[0]]
  return embeddings

sents = file_reader(training_data)
print(len(sents))
print(sents[0:10])

allWords, word2int, int2word = sents_processing(sents)
print(word2int)

# load embeding and build embeding dict
pre_embeddings = load_glove_embeddings()
embeddings = embedding_processing(word2int, pre_embeddings)

print("Vocal Size in all poems : ", len(word2int)) # 6435
VOCAB_SIZE = len(word2int)

132836
[['another', 'uae', 'space', 'object', 'is', 'ready', 'for', 'lift', 'off', 'right', 'on', 'the', 'heels', 'of', 'the', 'historic', 'launch', 'of', 'the', 'first', 'emirati', 'built', 'satellite'], ['mysat', '1', 'the', 'uae', 'student', 'built', 'nanosatellite', 'is', 'all', 'set', 'to', 'hit', 'the', 'skies', 'on', 'november', '15'], ['mysat', '1', 'an', 'educational', 'cubesat', 'built', 'by', '20', 'students', 'at', 'khalifa', 'university', 'in', 'abu', 'dhabi', 'with', 'yahsat', 'cooperation', 'is', 'due', 'to', 'be', 'launched', 'as', 'part', 'of', 'resupply', 'mission', 'to', 'the', 'international', 'space', 'station', 'iss', 'on', 'board', 'a', 'northrop', 'grumman', 'antares', 'launch', 'vehicle', 'from', 'us', 'on', '15', 'nov', '2018'], ['dr', 'mohammed', 'alahbabi', 'dralahbabi', 'november', '10', '2018', 'just', 'last', 'week', 'on', 'october', '29', 'the', 'country', 's', 'khalifasat', 'was', 'launched', 'into', 'space', 'from', 'japan', 'by', 'the', 'mohammed', 'b